In [48]:
import numpy as np
import zipfile
import cv2
from PIL import Image

class Network:
    def __init__(self, input_dim, hidden_layers, output_dim, dropout_ratio=0.2):
        self.num_stages = len(hidden_layers) + 1
        self.weight_matrices = []
        self.bias_vectors = []
        self.dropout_ratio = dropout_ratio
        layer_dimensions = [input_dim] + hidden_layers + [output_dim]

        # Initialize weight matrices and bias vectors
        for layer in range(self.num_stages):
            self.weight_matrices.append(
                np.random.randn(layer_dimensions[layer], layer_dimensions[layer + 1]) * np.sqrt(2 / layer_dimensions[layer])
            )
            self.bias_vectors.append(np.zeros((1, layer_dimensions[layer + 1])))

    def activation_relu(self, matrix):
        return np.maximum(0, matrix)

    def relu_gradient(self, matrix):
        return (matrix > 0).astype(float)

    def activation_softmax(self, matrix):
        exponential_values = np.exp(matrix - np.max(matrix, axis=1, keepdims=True))
        return exponential_values / np.sum(exponential_values, axis=1, keepdims=True)

    def dropout_masking(self, matrix):
        mask = np.random.binomial(1, 1 - self.dropout_ratio, size=matrix.shape) / (1 - self.dropout_ratio)
        return matrix * mask

    def forward_pass(self, input_data, is_training=True):
        # Flatten input if necessary
        if len(input_data.shape) > 2:  # If images are not flattened
            input_data = input_data.reshape(input_data.shape[0], -1)

        self.layer_outputs = [input_data]  # Store inputs as the first activation
        self.intermediate_z = []

        for layer in range(self.num_stages - 1):
            # Ensure dimensions are correct
            z_calc = np.dot(self.layer_outputs[-1], self.weight_matrices[layer]) + self.bias_vectors[layer]
            self.intermediate_z.append(z_calc)
            activation_output = self.activation_relu(z_calc)

            if is_training:
                activation_output = self.dropout_masking(activation_output)  # Dropout if training
            self.layer_outputs.append(activation_output)

        z_final = np.dot(self.layer_outputs[-1], self.weight_matrices[-1]) + self.bias_vectors[-1]
        self.intermediate_z.append(z_final)
        self.layer_outputs.append(self.activation_softmax(z_final))

        return self.layer_outputs[-1]

    def backward_pass(self, input_data, true_labels, predicted_output):
        sample_count = input_data.shape[0]
        self.gradient_weights = []
        self.gradient_biases = []

        delta = predicted_output - true_labels
        for layer in range(self.num_stages - 1, -1, -1):
            gradient_w = np.dot(self.layer_outputs[layer].T, delta) / sample_count
            gradient_b = np.sum(delta, axis=0, keepdims=True) / sample_count

            self.gradient_weights.insert(0, gradient_w)
            self.gradient_biases.insert(0, gradient_b)

            if layer > 0:
                delta_activation = np.dot(delta, self.weight_matrices[layer].T)
                delta = delta_activation * self.relu_gradient(self.intermediate_z[layer - 1])

    def parameter_update(self, learning_rate, l2_penalty):
        for layer in range(self.num_stages):
            self.weight_matrices[layer] -= learning_rate * (self.gradient_weights[layer] + l2_penalty * self.weight_matrices[layer])
            self.bias_vectors[layer] -= learning_rate * self.gradient_biases[layer]


def transform_image(image_data):
    reshaped_image = image_data.reshape(28, 28)

    # Apply random rotation
    if np.random.rand() < 0.5:
        rotation_angle = np.random.randint(-20, 20)
        reshaped_image = cv2.warpAffine(
            reshaped_image,
            cv2.getRotationMatrix2D((14, 14), rotation_angle, 1.0),
            (28, 28)
        )

    # Apply horizontal flipping
    if np.random.rand() < 0.5:
        reshaped_image = cv2.flip(reshaped_image, 1)

    # Horizontal shifting
    if np.random.rand() < 0.5:
        horizontal_shift = np.random.randint(-5, 5)
        reshaped_image = np.roll(reshaped_image, horizontal_shift, axis=1)

    # Vertical shifting
    if np.random.rand() < 0.5:
        vertical_shift = np.random.randint(-5, 5)
        reshaped_image = np.roll(reshaped_image, vertical_shift, axis=0)

    return reshaped_image.flatten()

def zip_content(archive_path):
    picture_set = []
    class_labels = []

    with zipfile.ZipFile(archive_path, "r") as archive:
        file_entries = archive.namelist()

        for entry in file_entries:
            directory_parts = entry.split('/')
            if len(directory_parts) > 1 and directory_parts[-1].endswith('.png'):
                category_name = directory_parts[-2]

                image_matrix = cv2.imdecode(
                    np.frombuffer(archive.read(entry), np.uint8), cv2.IMREAD_GRAYSCALE
                )
                if image_matrix is not None:
                    image_matrix = cv2.resize(image_matrix, (28, 28))  # Resize to 28x28
                    image_matrix = image_matrix / 255.0
                    picture_set.append(image_matrix.flatten())
                    class_labels.append(category_name)

        picture_set = np.array(picture_set)

        # Map text-based categories to numeric values
        class_to_index = {category: idx for idx, category in enumerate(sorted(set(class_labels)))}
        numeric_classes = np.array([class_to_index[cls] for cls in class_labels])

        return picture_set, numeric_classes, class_to_index

train_zip_path = 'Train (1).zip'
train_images, train_labels, label_map = zip_content(train_zip_path)

test_zip_path = 'Test (1).zip'
test_images, test_labels, _ = zip_content(test_zip_path)

def encode(labels, num_classes):
    labels = labels.astype(int)
    return np.eye(num_classes)[labels]

num_classes = len(label_map)

train_labels = encode(train_labels, num_classes)
test_labels = encode(test_labels, num_classes)

pixels = 784  # Update input size for 28x28 images
layers = [30]
output_size = num_classes
alpha = 0.02
iterations = 300
batch_size = 32
xlambda = 0.01
rate = 0.8

network = Network(pixels, layers, output_size)

for epoch in range(iterations):
    permutation = np.random.permutation(train_images.shape[0])
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]

    epoch_loss = 0
    for i in range(0, train_images.shape[0], batch_size):
        batch_X = train_images[i:i + batch_size]
        batch_y = train_labels[i:i + batch_size]

        batch_X = np.array([transform_image(img) for img in batch_X])

        output = network.forward_pass(batch_X)
        cross_entropy_loss = -np.mean(np.sum(batch_y * np.log(output + 1e-15), axis=1))
        l2_loss = (xlambda / 2) * sum(np.sum(np.square(w)) for w in network.weight_matrices)
        loss = cross_entropy_loss + l2_loss
        epoch_loss += loss

        network.backward_pass(batch_X, batch_y, output)
        network.parameter_update(alpha, xlambda)


In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

def evaluate_model(predictions, true_labels):
    # Get predicted classes by taking the index of the max probability
    predicted_classes = np.argmax(predictions, axis=1)

    # Get true class labels by taking the index of the max value in the one-hot encoded vector
    true_classes = np.argmax(true_labels, axis=1)

    # Accuracy
    accuracy = accuracy_score(true_classes, predicted_classes)

    # Precision, Recall, and F1-Score (per class, macro average)
    precision = precision_score(true_classes, predicted_classes, average='macro')
    recall = recall_score(true_classes, predicted_classes, average='macro')
    f1 = f1_score(true_classes, predicted_classes, average='macro')

    # Confusion Matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    return accuracy, precision, recall, f1, conf_matrix

# Calculate metrics after training
predictions = network.forward_pass(test_images, is_training=False)
accuracy, precision, recall, f1, conf_matrix = evaluate_model(predictions, test_labels)

print(f"Accuracy: {accuracy*100:.4f}%")
print(f"Precision (Macro): {precision*100:.4f}%")
print(f"Recall (Macro): {recall*100:.4f}%")
print(f"F1-Score (Macro): {f1*100:.4f}%")
print("Confusion Matrix:")
print(conf_matrix)



Accuracy: 91.1000%
Precision (Macro): 91.4322%
Recall (Macro): 91.1000%
F1-Score (Macro): 90.9983%
Confusion Matrix:
[[199   0   0   1   0]
 [  0 192   0   8   0]
 [  7   0 167   3  23]
 [  0  37   2 161   0]
 [  0   0   8   0 192]]
